<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Multilayer Perceptron</font></h1>

<h1><font color="#113D68" size=4>Desarrollo de una red neuronal para clasificar casos de DIABETES</font></h1>
<h1><font color="#113D68" size=4>Development of a neural network to classify cases of DIABETES</font></h1>


<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Ayelén Stefanía Andrade</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [1. Cargar datos](#section1)
* [2. Definir modelo](#section2)
* [3. Compilar modelo](#section3)
* [4. Ajustar modelo](#section4)
* [5. Evaluar modelo](#section5)
* [6. Hacer predicciones](#section6)


<a id="section1"></a>
# <font color="#004D7F" size=6>1. Cargar datos</font>

VAmos a cargar el archivo directamente usando la función `loadtxt()` de NumPy y establecer dividir las características y target.

In [105]:
#Source = https://www.kaggle.com/uciml/pima-indians-diabetes-database
import numpy as np
data = np.loadtxt('Datasets/diabetes.csv', delimiter = ',', skiprows=1)
x= data[:, 0:8]
y= data[:, 8]
y

array([1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
       1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0.,
       0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
       1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 0., 0., 0., 0.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Definir modelo</font>

Los modelos en Keras se definen como una secuencia de capas. Creamos un modelo **Secuencial** y agregamos capas de una en una hasta que estemos satisfechos con nuestra topología de red. 

Lo primero que debe hacer bien es asegurarse de que la capa de entrada tenga el número correcto de entradas. Esto se puede especificar al crear la primera capa con el argumento `input_dim` y establecerlo en 8. 

Pregunta, ¿Cómo sabemos la cantidad de capas que debemos usar y sus tipos? 


En sí el procedimiento a utilizar será:
1. Las capas completamente conectadas se definen mediante la clase `Dense`. Se puede especificar:
    * El número de neuronas en la capa como primer argumento y 
    * Especificar la función de activación usando el parámetro `activation`.
2. Usaremos la función de activación ReLu en las dos primeras capas y la función de activación Sigmoidea en la capa de salida. 
3. Usamos una función de activación sigmoidea en la capa de salida para garantizar que la salida de nuestra red esté entre 0 y 1. 

La siguiente figura proporciona una descripción de la estructura de la red.


<img src="images/2_redNeuronal.png" width="200" height="200" />

In [106]:
from keras.models import Sequential
from keras.layers import Dense

#define keras model
model = Sequential()
#capa 1
model.add(Dense(12, input_dim=8, activation = 'relu')) #son 12 neuronas, con 8 entradas
#capa 2
model.add(Dense(8, activation = 'relu')) #recibe automaticamente las entradas de la capa anterior, no es necesario poner input_dim
model.add(Dense(6, activation = 'relu')) 
model.add(Dense(4, activation = 'relu')) 
#capa 3 de salida
model.add(Dense(1, activation = 'sigmoid')) #fc de activacion sigmoidea para que clasifique entre 0 y 1 (problema de clasificacion binaria)


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Compilar modelo</font>

Ahora que el modelo está definido, podemos compilarlo. 

Al compilar, debemos especificar algunas propiedades adicionales requeridas al entrenar la red. 

Debemos especificar:
1. La función de pérdida que se utilizará para evaluar un conjunto de ponderaciones, 
2. El optimizador utilizado para buscar entre diferentes ponderaciones para la red y
3. Cualquier métrica opcional que nos gustaría recopilar e informar durante el entrenamiento. 
    * En este caso usaremos la **pérdida logarítmica**, que para un problema de clasificación binaria se define como `binary_crossentropy`. 
4. También usaremos el algoritmo de Gradiente Descendiente `adam` 
5. Finalmente, debido a que es un problema de clasificación nuestra métrica será el Accuracy.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Obtenga más información sobre el algoritmo de optimización de Adam en el artículo [_Adam: A Method for Stochastic Optimization_](https://arxiv.org/abs/1412.6980)

In [107]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section4"></a>
# <font color="#004D7F" size=6>4. Ajustar modelo</font>

Hemos definido nuestro modelo y lo hemos compilado listo para un cálculo eficiente. Ahora es el momento de ejecutar el modelo en algunos datos llamando a la función `fit()`.
* El proceso de entrenamiento se ejecutará para un número fijo de iteraciones usando el argumento `epochs`. 
* También podemos establecer el número de instancias que se evalúan antes de que se realice una actualización de peso en la red, denominada tamaño de batch, y se establece mediante el argumento `batch_size`. 

In [116]:
model.fit(x, y, epochs = 200, batch_size=26)

Epoch 1/200
30/30 [==============================] - 0s 2ms/step - loss: 0.4279 - accuracy: 0.7943
Epoch 2/200
30/30 [==============================] - 0s 2ms/step - loss: 0.4304 - accuracy: 0.7891
Epoch 3/200
30/30 [==============================] - 0s 2ms/step - loss: 0.4279 - accuracy: 0.7904
Epoch 4/200
30/30 [==============================] - 0s 2ms/step - loss: 0.4420 - accuracy: 0.7812
Epoch 5/200
30/30 [==============================] - 0s 2ms/step - loss: 0.4349 - accuracy: 0.7917
Epoch 6/200
30/30 [==============================] - 0s 2ms/step - loss: 0.4366 - accuracy: 0.7760
Epoch 7/200
30/30 [==============================] - 0s 2ms/step - loss: 0.4304 - accuracy: 0.7995
Epoch 8/200
30/30 [==============================] - 0s 2ms/step - loss: 0.4298 - accuracy: 0.7904
Epoch 9/200
30/30 [==============================] - 0s 2ms/step - loss: 0.4290 - accuracy: 0.7865
Epoch 10/200
30/30 [==============================] - 0s 2ms/step - loss: 0.4345 - accuracy: 0.7865
Epoch 11/

30/30 [==============================] - 0s 1ms/step - loss: 0.4170 - accuracy: 0.8034
Epoch 84/200
30/30 [==============================] - 0s 2ms/step - loss: 0.4261 - accuracy: 0.7995
Epoch 85/200
30/30 [==============================] - 0s 1ms/step - loss: 0.4127 - accuracy: 0.8021
Epoch 86/200
30/30 [==============================] - 0s 2ms/step - loss: 0.4230 - accuracy: 0.7930
Epoch 87/200
30/30 [==============================] - 0s 1ms/step - loss: 0.4170 - accuracy: 0.7917
Epoch 88/200
30/30 [==============================] - 0s 1ms/step - loss: 0.4383 - accuracy: 0.7799
Epoch 89/200
30/30 [==============================] - 0s 1ms/step - loss: 0.4284 - accuracy: 0.7982
Epoch 90/200
30/30 [==============================] - 0s 1ms/step - loss: 0.4288 - accuracy: 0.7930
Epoch 91/200
30/30 [==============================] - 0s 1ms/step - loss: 0.4236 - accuracy: 0.7956
Epoch 92/200
30/30 [==============================] - 0s 1ms/step - loss: 0.4235 - accuracy: 0.8047
Epoch 93/200


30/30 [==============================] - 0s 1ms/step - loss: 0.4164 - accuracy: 0.8008
Epoch 164/200
30/30 [==============================] - 0s 1ms/step - loss: 0.4089 - accuracy: 0.8008
Epoch 165/200
30/30 [==============================] - 0s 1ms/step - loss: 0.4207 - accuracy: 0.7917
Epoch 166/200
30/30 [==============================] - 0s 1ms/step - loss: 0.4144 - accuracy: 0.8034
Epoch 167/200
30/30 [==============================] - 0s 1ms/step - loss: 0.4146 - accuracy: 0.7956
Epoch 168/200
30/30 [==============================] - 0s 1ms/step - loss: 0.4109 - accuracy: 0.7982
Epoch 169/200
30/30 [==============================] - 0s 1ms/step - loss: 0.4177 - accuracy: 0.7917
Epoch 170/200
30/30 [==============================] - 0s 1ms/step - loss: 0.4203 - accuracy: 0.7969
Epoch 171/200
30/30 [==============================] - 0s 1ms/step - loss: 0.4176 - accuracy: 0.7995
Epoch 172/200
30/30 [==============================] - 0s 1ms/step - loss: 0.4127 - accuracy: 0.7995
Epoc

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section5"></a>
# <font color="#004D7F" size=6>5. Evaluar modelo</font>

Podemos evaluar tu modelo en tu conjunto de datos de entrenamiento usando la función de `evaluation()` y pasar la misma entrada y salida que usaste para entrenar el modelo.

In [117]:
_,accuracy = model.evaluate(x,y) # _ significa ignorar los otros datos

24/24 [==============================] - 0s 1ms/step - loss: 0.4111 - accuracy: 0.8034


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section6"></a>
# <font color="#004D7F" size=6>6. Hacer predicciones</font>

Podemos llamar a la función `predict()` para predecir clases directamente, por ejemplo:

In [121]:
predictions = model.predict(x)

predictions = np.round(predictions).astype(int)
predictions

array([[1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
    

Veamos un ejemplo

In [136]:
print('Existen {} casos de diabetes en esta base de datos. \n'.format(contador_casos_diabetes))
print('There are {} diabetes cases in the database analysed.\n'.format(contador_casos_diabetes))

print('\nPredictions/Predicciones:')
for i in range(10):
    print('%s predicción -->  %d (real %d) ' % (x[i].tolist(), predictions[i], y[i]))
contador_casos_diabetes = 0
for i in range(len(predictions)):
    contador_casos_diabetes += 1


Existen 768 casos de diabetes en esta base de datos. 

There are 768 diabetes cases in the database analysed.


Predictions/Predicciones:
[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0] predicción -->  1 (real 1) 
[1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0] predicción -->  0 (real 0) 
[8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0] predicción -->  1 (real 1) 
[1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167, 21.0] predicción -->  0 (real 0) 
[0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0] predicción -->  1 (real 1) 
[5.0, 116.0, 74.0, 0.0, 0.0, 25.6, 0.201, 30.0] predicción -->  0 (real 0) 
[3.0, 78.0, 50.0, 32.0, 88.0, 31.0, 0.248, 26.0] predicción -->  0 (real 1) 
[10.0, 115.0, 0.0, 0.0, 0.0, 35.3, 0.134, 29.0] predicción -->  0 (real 0) 
[2.0, 197.0, 70.0, 45.0, 543.0, 30.5, 0.158, 53.0] predicción -->  1 (real 1) 
[8.0, 125.0, 96.0, 0.0, 0.0, 0.0, 0.232, 54.0] predicción -->  0 (real 1) 


Ejecutar el ejemplo no muestra la barra de progreso como antes, ya que hemos establecido el argumento `verbose` en 0. 

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>